In [1]:
%load_ext autoreload
%autoreload 2

# Settings, Directory Specs, and Imports

In [2]:
dir_read = '../data/preprocessed/' # contains csv file you're reading from. See README for the format
filename_Xy = 'Xy_2020_06_20_2258.csv'
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, confusion_matrix, plot_confusion_matrix, f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, learning_curve, train_test_split, ShuffleSplit, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import check_random_state
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import streamlit as st
import pickle
from datetime import datetime
from sklearn.model_selection._split import _BaseKFold
from sklearn.model_selection._split import _RepeatedSplits
from collections import defaultdict
from collections import Counter
import sys
sys.path.insert(1, '../src')
from utils import *

matplotlib.rcParams.update({'font.size': 22})

# Defining X and y, train/val and test

In [3]:
X, y, Xy, groups, vars_categ, vars_cont = createXy(dir_read, filename_Xy)
X.head()

There are 21 categorical features
There are 9 continuous features


,age,admissionweight,admissionheight,bmi,verbal,motor,eyes,visitnumber,heartrate,gender_Female,...,metastaticcancer,leukemia,immunosuppression,cirrhosis,activetx,ima,midur,oobventday1,oobintubday1,diabetes
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141168,70,84.3,152.4,36.295906,5,6,4,1,125.052830,1,...,0,0,0,0,1,0,0,0,0,0
141194,68,73.9,180.3,22.732803,4,6,3,1,86.860627,0,...,0,0,0,0,0,0,0,0,0,1
141197,71,102.1,162.6,38.617545,5,6,4,1,97.307692,0,...,0,0,0,0,0,0,0,0,0,0
141203,77,70.2,160.0,27.421875,1,3,1,1,91.543554,1,...,0,0,0,0,1,0,0,1,0,1
141208,25,95.3,172.7,31.952749,5,6,3,1,77.817460,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Split into 80/20 train&val/test using StratifiedGroupKFold()
cv, X_trainval, y_trainval, X_test, y_test = train_test_split_StratifiedGroupKFold(X, y, groups, 5, 1)

X_trainval has 116993 unique patientstayids
X_test has 29249 unique patientstayids


# Feature Selection

In [5]:
from sklearn.feature_selection import RFECV

# For logistic regression, scale the data
scaler = MinMaxScaler()
X_trainval_sc = np.concatenate([scaler.fit_transform(X_trainval[vars_cont]), \
                             X_trainval[vars_categ].to_numpy()], axis=1)
X_test_sc = np.concatenate([scaler.transform(X_test[vars_cont]), \
                             X_test[vars_categ].to_numpy()], axis=1)

# Create Logistic Regression classifier
clf = LogisticRegression(class_weight='balanced')

# Perform recursive feature elimination
rfecv = RFECV(estimator=clf,
              step=1,
              cv=StratifiedKFold(n_splits=5),
              n_jobs=4,
              scoring='roc_auc')
rfecv.fit(X_trainval_sc, y_trainval)
print("Optimal number of features : %d" % rfecv.n_features_)

# Get the names of features to keep
feature_names = X.columns
feature_importance = list(zip(feature_names, rfecv.support_))
new_features = []
for key,value in enumerate(feature_importance):
    if(value[1]) == True:
        new_features.append(value[0])
        
print(new_features)

/Users/rachellehorwitz/opt/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/rachellehorwitz/opt/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    

Optimal number of features : 8
['age', 'admissionweight', 'visitnumber', 'heartrate', 'aids', 'ima', 'midur', 'oobintubday1']


In [6]:
# Use only the important features
X_trainval_imp = X_trainval_sc[:, rfecv.support_]
X_test_imp = X_test_sc[:, rfecv.support_]

# Hyperparameter Tuning

In [7]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix

# Create base classifier
clf = LogisticRegression(class_weight='balanced')

# StratifiedGroupKFold takes a long time, so for tuning hyperparameters, just use StratifiedKFold
skf = StratifiedKFold(n_splits=10)

# Create grid of hyperparameters
hyperparam_grid = {'penalty': ['l1', 'l2'], \
                   'C': np.logspace(-6, 3, 10), \
                   #'C': [0.001, 0.01], \
                   'fit_intercept': [True, False]}

# Perform grid search
grid_search = GridSearchCV(clf,
                           scoring='roc_auc',
                           param_grid=hyperparam_grid,
                           refit=True,
                           cv=skf,
                           return_train_score=True,
                           n_jobs=-1)
grid_search.fit(X_trainval_imp, y_trainval)

print('The best parameters from grid search are:')
print(grid_search.best_params_)

The best parameters from grid search are:
{'C': 0.01, 'fit_intercept': True, 'penalty': 'l2'}


In [8]:
# Create classifier with tuned hyperparameters
clf_w_best_params = LogisticRegression(class_weight = 'balanced', \
                                       C = grid_search.best_params_['C'], \
                                       fit_intercept = grid_search.best_params_['fit_intercept'], \
                                       penalty = grid_search.best_params_['penalty'])
clf_w_best_params.fit(X_trainval_imp,
                      y_trainval)  # Fit classifier to train & validation data

y_pred = clf_w_best_params.predict(
    X_test_imp)  # binary classification of test data
y_probs = clf_w_best_params.predict_proba(
    X_test_imp)[:, 1]  # probabilities of test data

y_probs_TRAIN = clf_w_best_params.predict_proba(
    X_trainval_imp)[:, 1]  # probabilities of training data

# Pickle model and scaler
model_and_scaler = {'model': clf_w_best_params,
                    'scaler': scaler, \
                    'feature_mask': rfecv.support_, \
                    'vars_cont': vars_cont,
                    'vars_categ': vars_categ,
                    'y_probs_TRAIN_mean': y_probs_TRAIN.mean()}
pickle.dump(model_and_scaler, \
            open('../models/model_scaler_logRegr_featsel' + now_to_str() + '.pickle', 'wb'))

# Print results
print('Test AUC is ' + str(roc_auc_score(y_test, y_probs)))
print('Min. probability is {:.4f}'.format(y_probs.min()))
print('Mean probability is {:.4f}'.format(y_probs_TRAIN.mean()))
print('Max. probability is {:.4f}'.format(y_probs.max()))
print('--------------')
print('Train AUC is ' + str(roc_auc_score(y_trainval, y_probs_TRAIN)))
print('Min. probability is {:.4f}'.format(y_probs_TRAIN.min()))
print('Max. probability is {:.4f}'.format(y_probs_TRAIN.max()))

Test AUC is 0.7284276872726011
Min. probability is 0.0407
Mean probability is 0.4376
Max. probability is 0.9043
--------------
Train AUC is 0.7244975269897372
Min. probability is 0.0500
Max. probability is 0.9173


In [9]:
%matplotlib qt

# Plot 
plt.figure()
plt.hist(y_probs, bins=20)
plt.xlabel('Probability of VTE')
plt.ylabel('Number of patient stays')
plt.tight_layout()
print('Average probability of VTE is {:.3f}'.format(y_probs.mean()))

Average probability of VTE is 0.437


In [10]:
# Plot ROC curve
# Code modified from https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-imbalanced-classification/#:~:text=An%20ROC%20curve%20(or%20receiver,True%20Positive%20Rate%20(y).
noskill_probs = [0 for _ in range(len(y_test))]
ns_fpr, ns_tpr, _ = roc_curve(y_test, noskill_probs)
lr_fpr, lr_tpr, threshold_array = roc_curve(
    y_test,
    clf_w_best_params.predict_proba(X_test_imp)[:, 1])

y_pred_new = lr_tpr > 0

logisticRegr_auc = roc_auc_score(y_test, y_probs)
print('Logistic: ROC AUC=%.3f' % (logisticRegr_auc))

plt.figure()
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, linestyle='--')
plt.plot(lr_fpr, lr_tpr, marker='.', label='Logistic Regr.')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# # show the legend
#plt.legend()
plt.tight_layout()
# # show the plot
plt.show()
plt.title('Logistic Regression: ROC AUC=%.3f' % (logisticRegr_auc))
plt.tight_layout()

Logistic: ROC AUC=0.728


In [11]:
# Determine which threshold to use. After consulting with Sonam Dilwali, M.D., Ph.D.,
# a reasonable threshold is TRP ~ 0.75, FPR ~0.4
threshold_array[(lr_tpr>0.75) & (lr_fpr<0.39)]

array([0.44075921])

# Feature Weights

In [12]:
# Get feature weights and put into dataframe
mydict = {'feature': new_features, 'coef': list(clf_w_best_params.coef_.reshape(-1,1).flatten())}
features_weights = pd.DataFrame(mydict)
features_weights = features_weights.assign(abs_weight=np.abs(features_weights['coef']))
features_weights.head()

,feature,coef,abs_weight
0,age,0.525954,0.525954
1,admissionweight,1.508892,1.508892
2,visitnumber,1.177785,1.177785
3,heartrate,3.439470,3.439470
4,aids,-0.276730,0.276730


In [13]:
# Wrangle feature importances
fw = features_weights.sort_values(by='abs_weight', ascending=False)
fw = fw[['feature', 'coef']]
fw = fw.set_index('feature')
fw = fw.rename({'heartrate': 'Avg. HR\nDuring\nFirst 24\n Hours', \
               'ima': 'Internal \nMammary \nArtery Graft', \
               'admissionweight': 'Admission\nWeight', \
               'visitnumber': 'Visit Number', \
               'oobintubday1': 'Intubated', \
               'midur': 'Heart Attack \nwithin 6 Mths', \
               'age': 'Age', \
               'aids': 'AIDS'})
fw

,coef
feature,
Avg. HR\nDuring\nFirst 24\n Hours,3.439470
Internal \nMammary \nArtery Graft,-1.509630
Admission\nWeight,1.508892
Visit Number,1.177785
Intubated,1.170120
Heart Attack \nwithin 6 Mths,-0.672236
Age,0.525954
AIDS,-0.276730


In [14]:
# Create plot of feature importances
fig = plt.figure(figsize=(17.5, 5))
matplotlib.rcParams.update({'font.size': 16})
plt.bar(np.arange(0, fw.shape[0]), fw['coef'].values, tick_label=list(fw.index))
plt.axhline(y=0, color='black')
plt.xticks(rotation=0)
plt.xlabel('')
plt.ylabel('Coefficient')
plt.tight_layout()